In [ ]:
pip install request

ERROR: Could not find a version that satisfies the requirement request (from versions: none)
ERROR: No matching distribution found for request


# Задание 1

Задача: Создать чат бота для получения информации об исследованиях космоса

Описание: Создайте комплексное приложение командной строки, которое будет использоваться в качестве панели управления исследованиями космоса. Данное приложение будет обращаться к https://api.nasa.gov/ для предоставления пользователям набора информации о космосе, включая:

- Астрономическая картинка дня (APOD): Отображение APOD с пояснениями к нему.
- Фотографии с марсохода: позволяет пользователям выбирать и фильтровать фотографии с марсохода по дате и типу камеры.
- Объекты, сближающиеся с Землей (ОСЗ): Поиск и отображение информации об объекте, сближающихся с Землей, на определенную дату, включая их размеры и потенциальную опасность.
- Данные о космической погоде: Отображают последние данные о космической погоде, включая солнечные вспышки и геомагнитные бури.
Приложение должно позволять пользователям ориентироваться в этих функциях, корректно обрабатывать ошибки и обеспечивать удобство работы.

Требования:
- Пользовательский ввод: Приложение должно предложить пользователю ввести данные, чтобы выбрать, какую функцию он хочет изучить.
- Проверка данных: Убедитесь, что пользовательские данные (например, даты) проверены.
- Обработка ошибок: Корректно обрабатывайте ошибки API и неверные ответы.
- Представление данных: Представляйте данные в четкой и организованной форме.
- Опция выхода: позволяет пользователям выходить из приложения в любое время.

key: eimFDY02wAvM4AJhCNQrNuIdEqkGFPz4bBrmikOE

In [ ]:
# @title Текст заголовка по умолчанию
import requests as req
from PIL import Image
from IPython.display import display
from datetime import datetime


def send_request(url):
    response = req.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print('Response code:', response.status_code)
        None


def main():
    #Картинка дня
    def image_today(data):
        img_url = data['url']
        response = req.get(img_url, stream=True).raw
        img = Image.open(response)
        display(img)
        title = data['title']
        body = data['explanation']
        return f'{title}\n \n{body}'


    #Картинка с Марсохода
    def image_mars(data):
        img_mars = data['photos'][0]['img_src']
        response = req.get(img_mars, stream=True).raw
        img = Image.open(response)
        return display(img)


    #Данные о небесных телах
    def neows(data, date):
        dict_neows = {}
        for i in range(len(data['near_earth_objects'][str(date)])):
            dict_neows[str(data['near_earth_objects'][str(date)][i]['name'])] = ['Миниимальный радиус(м):',data['near_earth_objects'][str(date)][i]\
              ['estimated_diameter']['meters']['estimated_diameter_min'], \
              'Максимальный радиус(м):', data['near_earth_objects'][str(date)]\
               [i]['estimated_diameter']['meters']['estimated_diameter_max'], \
              'Потенциальная опасность:', data['near_earth_objects'][str(date)]\
               [i]['is_potentially_hazardous_asteroid']]
        return dict_neows


    #Данные о погоде в космосе
    def space_weather():
        date = datetime.today()
        day = date.day
        month = date.month
        year = date.year
        url_all = f'https://api.nasa.gov/DONKI/notifications?&type=all&api_key=eimFDY02wAvM4AJhCNQrNuIdEqkGFPz4bBrmikOE'
        message_all = send_request(url_all)[1]['messageBody']
        url_solar = f'https://api.nasa.gov/DONKI/FLR?startDate={year}-{month}-{str(int(day)-3)}&endDate={year}-{month}-{day}&api_key=eimFDY02wAvM4AJhCNQrNuIdEqkGFPz4bBrmikOE'
        message_solar = {}
        number = 0
        for i in range(len(send_request(url_solar))):
            number+=1
            message_solar[str(number)]=['Время начала:', send_request(url_solar)[i]['beginTime'], 'Время конца:', send_request(url_solar)[i]['endTime'],'Класс-тип:', send_request(url_solar)[i]['classType']]
        url_geomagnetic = f'https://api.nasa.gov/DONKI/GST?startDate={year}-{month}-{str(int(day)-3)}&endDate={year}-{month}-{day}&api_key=eimFDY02wAvM4AJhCNQrNuIdEqkGFPz4bBrmikOE'
        message_geomagnetic = {}
        number1 = 0
        for i in range(len(send_request(url_geomagnetic))):
            number1+=1
            message_geomagnetic[str(number1)] = ['Время начала:',send_request(url_geomagnetic)[i]['startTime'], 'Сила бури:', send_request(url_geomagnetic)[i]['allKpIndex'][0]['kpIndex']]
        return print(f'Общие данные: {message_all} \n\nСолнечные вспышки: {message_solar} \n\nМагнитные бури: {message_geomagnetic}')


    query = ''
    while 'стоп' not in query:
        query = list(map(str,input().split()))
        if '1' in query:
            data = send_request('https://api.nasa.gov/planetary/apod?api_key=eimFDY02wAvM4AJhCNQrNuIdEqkGFPz4bBrmikOE')
            return image_today(data)

        if '2' in query:
            rover_in = input('Выберите Марсоход: \n Curiosity \n Opportunity \n Spirit \n')
            type_cam = input('Выберите тип камеры: \n FHAZ \n RHAZ \n MAST \n')
            date_earth = input('Введите дату в формате XXXX-YY-ZZ: \n XXXX-год \n YY-месяц \n ZZ-день \n')
            url = f'https://api.nasa.gov/mars-photos/api/v1/rovers/{rover_in}/photos?earth_date={date_earth}&camera={type_cam}&api_key=eimFDY02wAvM4AJhCNQrNuIdEqkGFPz4bBrmikOE'
            data = send_request(url)
            return image_mars(data)

        if '3' in query:
            date_begin = input('Введите дату начала в формате XXXX-YY-ZZ: \n XXXX-год \n YY-месяц \n ZZ-день \n')
            date_end = input('Введите дату окончания в формате XXXX-YY-ZZ: \n XXXX-год \n YY-месяц \n ZZ-день \n')
            url = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={date_begin}&e\
            nd_date={date_end}&api_key=eimFDY02wAvM4AJhCNQrNuIdEqkGFPz4bBrmikOE'
            data = send_request(url)
            return neows(data, date_begin)

        if '4' in query:
            return space_weather()


if __name__ == '__main__':
    main()

4
Общие данные: ## Community Coordinated Modeling Center Database Of Notifications, Knowledge, Information ( CCMC DONKI )
## Message Type: Auto-generated Space Weather Notification - Flare intensity has crossed the threshold of M5.0
##
## Message Issue Date: 2024-11-10T12:10:52Z
## Message ID: 20241110-AL-001
##
## Disclaimer: NOAA's Space Weather Prediction Center is the United States Government official source for space weather forecasts. This "Experimental Research Information" consists of preliminary NASA research products and should be interpreted and used accordingly.


## Summary:
 

Significant flare detected by GOES.  Flare intensity has crossed the threshold of M5.0.

Flare M5.0 crossing time: 2024-11-10T12:03Z. 

Increased energetic proton fluxes possible. Updates on this event will be provided when available.

## Notes: 



 

Солнечные вспышки: {'1': ['Время начала:', '2024-11-07T01:15Z', 'Время конца:', '2024-11-07T01:39Z', 'Класс-тип:', 'M2.5'], '2': ['Время начала:', '2

# Задание 2

Описание задачи

Цель этой задачи - создать скрипт на Python, который взаимодействует с API Чикагского института искусств (https://api.artic.edu/docs/) для извлечения и отображения произведений искусства. Скрипт должен позволять пользователям просматривать работы по страницам, фильтровать их по имени художника и просматривать подробную информацию о выбранных произведениях искусства. Ниже приведены требования и функциональные возможности, которые необходимо реализовать:

Требования:
Извлекать произведения искусства:

- Создайте функцию, которая извлекает список произведений искусства из API Чикагского института искусств.
Функция должна принимать параметр page для разбивки на страницы и возвращать список произведений искусства вместе с информацией о разбивке на страницы.
Фильтровать произведения искусства:

- Реализуйте функцию, которая фильтрует список произведений искусства на основе имени указанного художника. Функция должна возвращать список работ, которые соответствуют имени художника (без учета регистра).
Отображать подробную информацию об оформлении:

- Напишите функцию, которая отображает названия работ для пользователя и позволяет ему выбрать одну из них, введя соответствующий номер.
После выбора функция должна отображать подробную информацию о выбранном произведении, включая название, исполнителя, дату и носитель.
Разбивка на страницы и взаимодействие с пользователем:

- Создайте основную функцию, которая управляет выборкой произведений и взаимодействием с пользователем.

Разрешите пользователям перемещаться по страницам с произведениями искусства, выполнять фильтрацию по исполнителю или выходить из программы.

Если страниц с произведениями искусства несколько, укажите варианты перехода к следующей странице, предыдущей странице, фильтрации по исполнителю или выхода из программы.

In [ ]:
import requests as req

def send_request(url):
    response = req.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print('Response code:', response.status_code)
        None


def main():
    def list_art(data):
        number = 0
        dict_art = {}
        list_art = []

        for i in range(len(data['data'])):
            list_art.append(data['data'][i]['title'])

        for j in list_art:
            number+=1
            dict_art[str(number)] = j
        print(dict_art)


    def sort_list(data):
        dict_art = {}
        list_name_artist = []
        list_art = []
        number = 0
        for i in range(len(data['data'])):
            list_name_artist.append(data['data'][i]['artist_display'])
        list_name_artist = sorted(list_name_artist)

        for name in list_name_artist:
            for l in range(len(data['data'])):
                if name == data['data'][l]['artist_display']:
                    list_art.append(data['data'][l]['title'])

        for j in list_art:
            number+=1
            dict_art[str(number)] = j
        print(dict_art)


    def information_art(dict_art, number, data):
        information = {}
        for k in dict_art.keys():
            if k == str(number):
                key = k
        print(data['data'][int(number)-1]['title'])
        print(data['data'][int(number)-1]['artist_display'])
        print(data['data'][int(number)-1]['date_display'])

    query = input()

    if '1' in query:
        page = input('Введите страницу: ')
        data = send_request(f'https://api.artic.edu/api/v1/artworks?page={page}')
        print(list_art(data))

    if '2' in query:
        page = input('Введите страницу: ')
        data = send_request(f'https://api.artic.edu/api/v1/artworks?page={page}')
        print(sort_list(data))

    if '3' in query:
        page = input('Введите страницу: ')
        data = send_request(f'https://api.artic.edu/api/v1/artworks?page={page}')
        data_1 = list_art(data)
        print(data_1)
        number = input('Введите номер произведения: ')
        print(information_art(data_1, number, data))


if __name__ == '__main__':
    main()

1
Введите страницу: 2
{'1': 'The USSR in Construction (SSSR na stroike)', '2': 'Walt Whitman in Camden', '3': 'We Live [Žijeme]', '4': 'Pásmo [Zone or Reel]', '5': 'Lef (Left Front of the Arts)', '6': 'Emilie prichází ke mne ve snu (Emilie Comes To Me In A Dream)', '7': 'Koruna (Crown)', '8': "A.I.Z. (Workers' Illustrated Magazine)", '9': 'The Industry of Socialism (Industriia sotsializma)', '10': 'Union of Soviet Socialist Republics:  Catalogue of the Soviet Pavilion at the International Press Exhibition, Cologne ("Pressa") [Union der Sozialistischen Sowjet-Republiken:  Katalog des Sowjet-Pavillons auf der Internationalen Presse-Ausstellung, Köln ("Pressa")', '11': "Ubu cocu, restitué en son intégrité tel qu'il a été représenté par les marionnettes du Théâtre des Phynances: Cinq actes", '12': 'Le surmâle: Roman moderne'}
None


'1'

# Задание 3

Задача: Создать программу по управлению портфелем криптовалют

Цель: Создать скрипт на Python, который извлекает цены на криптовалюты в режиме реального времени, позволяет пользователям управлять портфелем криптовалют, вычисляет общую стоимость портфеля, отслеживает изменения цен и предоставляет исторические данные о ценах для анализа.

Требования:
Получение текущих цен на криптовалюты:

Используйте https://docs.coingecko.com/ для получения актуальных цен на список криптовалют.

Управление портфелем:

- Позволяет пользователю создавать портфель криптовалют и управлять им, указывая количество каждой криптовалюты, которой он владеет.
- Расчитывает общую стоимость портфеля в указанной фиатной валюте (например, долларах США).

Отслеживание изменения цен:

- Отображение процентного изменения цены для каждой криптовалюты в портфеле за последние 24 часа.
- Выделите все криптовалюты, стоимость которых значительно увеличилась или снизилась.

Поиск исторических данных о ценах:

- Получение исторических данных о ценах на указанную криптовалюту за последнюю неделю.
- Предоставьте пользователю возможность визуализировать эти данные в простом текстовом формате (например, цены за день).

Взаимодействие с пользователем:

- Реализуйте интерфейс командной строки для ввода данных пользователем.
- Предоставьте опции для получения текущих цен, управления портфелем, просмотра изменений цен или анализа исторических данных.


In [ ]:
def requests as req

def send_request(url):
    responce = req.get(url)
    if response.status_code == 200:
        return responce.json()
    else:
        print('Responce code:', responce.status_code)
        None



# Дополнительно: Задание 4

Задание 4: Проектное

Вам необходимо самостоятельно найти откртое API предоставляющее информацию в открытом доступе и реализовать собственный проект!


Критерии приемки результата:

- Проект включает в себя не менее 5 возможостей для пользователя
- Проект позволяет использовать все возможности проекта пользователю при помощи взаимодействия через коммандную строку
- Проект работает с открытым API (это значит что при проверке вашей работы преподавателем, преподавателю необходимо просто запустить ячейку с кодом вашего проекта и она будет работать без дополнительных манипуляции)
- Проект должен обязательно включать в себя ряд используемых конструкции:
    - Функции
    - Условные конструкции
    - Ввод/вывод
    - Словари/Списки
- Допускается использование библиотек:
    - requests
    - datetime
    - random

**Здесь добавьте описание вашего проекта**

In [ ]:
#  А здесь код